# **Análisis de la Expectativa de Vida Utilizando la Regresión Lineal**
La expectativa de vida es una medida estadística que indica cuántos años de vida, en promedio, puede esperar vivir un individuo, dadas ciertas condiciones o factores.

**DATA: https://raw.githubusercontent.com/JuanCarlosCS19/ML_Toulouse_ENE_2024/main/Clase1/Expectativa_Vida_Practica1.csv**

## Descripción de columnas(variables)

En este ejercicio, exploraremos un conjunto de datos que contiene información sobre la expectativa de vida en diferentes países y cómo se relaciona con varios factores de salud y socioeconómicos. Las variables en este conjunto de datos son las siguientes:


El objetivo de este ejercicio es utilizar estas variables para analizar y predecir la expectativa de vida en diferentes países. Esto implicará la limpieza y exploración de los datos, seguido de la creación y evaluación de un modelo de regresión lineal.

1. Country: El país al que pertenecen los datos.
2. Year: El año en que se recopilaron los datos.
3. Status: El estado de desarrollo del país (desarrollado o en desarrollo).
4. Life expectancy: La expectativa de vida promedio en años.
5. Adult Mortality: La tasa de mortalidad de adultos por cada 1000 habitantes.
6. Infant deaths: Número de muertes infantiles por cada 1000 habitantes.
7. Alcohol: Consumo de alcohol per cápita (15+), en litros de alcohol puro.
8. Percentage expenditure: Gasto en salud como porcentaje del Producto Interno Bruto per cápita.
9. Hepatitis B: Cobertura de inmunización contra la Hepatitis B (HepB) en porcentaje de niños de 1 año.
10. Measles: Número de casos reportados de sarampión por cada 1000 habitantes.
11. BMI: Promedio de Índice de Masa Corporal de toda la población.
12. Under-five deaths: Número de muertes de niños menores de cinco años por cada 1000 habitantes.
13. Polio: Cobertura de inmunización contra la poliomielitis (Pol3) en porcentaje de niños de 1 año.
14. Total expenditure: Gasto general del gobierno en salud como porcentaje del gasto total del gobierno.
15. Diphtheria: Cobertura de inmunización DTP3 en porcentaje de niños de 1 año.
16. HIV/AIDS: Muertes por cada 1 000 habitantes de 0 a 4 años debido al VIH/SIDA.
17. GDP: Producto Interno Bruto per cápita (en USD).
18. Population: Población del país.
19. Thinness 1-19 years: Prevalencia de delgadez entre niños y adolescentes de 10 a 19 años (%).
20. Thinness 5-9 years: Prevalencia de delgadez entre niños de 5 a 9 años (%).
21. Income composition of resources: Índice de Desarrollo Humano en términos de composición de ingresos de los recursos (índice que va de 0 a 1).
22. Schooling: Número promedio de años de escolarización.



In [ ]:
!pip install shap

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from xgboost import plot_importance
import shap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# ¿Que argumento nos falta?
df = pd.read_csv("https://raw.githubusercontent.com/JuanCarlosCS19/ML_Toulouse_ENE_2024/main/Clase1/Expectativa_Vida_Practica1.csv",sep=";")
df

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.columns

## Análisis Exploratorio de algunas variables(Este es un ejemplo pero ustedes pueden tomar mas variables o menos variables)

---



In [ ]:
# Visualización de la distribución de la variable objetivo (Life expectancy)
sns.histplot(df['Life expectancy '].dropna(), kde=True)
plt.title('Distribución de Life expectancy')
plt.show()

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
nan_percentage = df.isnull().mean() * 100

# Crear un DataFrame con la información
nan_info = pd.DataFrame({
    'Columna': nan_percentage.index,
    'Porcentaje de NA': nan_percentage.values
})

# Ordenar por el porcentaje descendente
nan_info = nan_info.sort_values(by='Porcentaje de NA', ascending=False)

# Mostrar el resultado
print(nan_info)

df = df.dropna()


In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(x='Status', y='Life expectancy ', data=df)
plt.title('Life Expectancy por Estado')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='GDP', y='Life expectancy ', data=df)
plt.title('Life Expectancy vs. GDP')
plt.xlabel('GDP')
plt.ylabel('Life Expectancy')
plt.show()

In [ ]:
selected_columns = ['Life expectancy ', 'Adult Mortality', 'Alcohol', ' BMI ', ' HIV/AIDS']
sns.pairplot(df[selected_columns].dropna())
plt.suptitle('Pairplot de Variables Seleccionadas', y=1.02)
plt.show()

In [ ]:
correlation_matrix = df[['Life expectancy ', 'Adult Mortality', 'Alcohol', ' BMI ', ' HIV/AIDS']].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación para Variables Seleccionadas')
plt.show()

## USANDO XGBOOST

In [ ]:
import xgboost as xgb
# Selecciona las variables de interés
selected_columns = ['Life expectancy ', 'Adult Mortality', 'Alcohol', ' BMI ', ' HIV/AIDS']
df_selected = df[selected_columns].dropna()

# División de datos en conjuntos de entrenamiento y prueba
X = df_selected.drop('Life expectancy ', axis=1)
y = df_selected['Life expectancy ']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuración del modelo XGBoost
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse'}
num_rounds = 100

# Convertir datos a formato DMatrix para XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Entrenar el modelo
model = xgb.train(params, dtrain, num_rounds)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(dtest)

# Calcular el coeficiente de determinación ajustado (R^2 ajustado)
n = X_test.shape[0]  # Número de observaciones en el conjunto de prueba
k = X_test.shape[1]  # Número de predictores
r2 = r2_score(y_test, y_pred)
r2_adj = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

# Imprimir métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')
print(f'R2 Adjusted: {r2_adj}')

# Visualización de las predicciones contra los valores reales
plt.scatter(y_test, y_pred)
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs. Valores reales (XGBoost)')
plt.show()
